# Imagenes

## Conceptos fundamentales de Docker: imágenes

Son como docker intenta solucionar la construccion y distribucion de software. Las imagenes son plantillas que docker usa como insumo para crear contenedores.

Una imagen es una pieza de software empaquetada,  de forma liviana que tiene todo lo necesario para que un contenedor pueda ejecutarse satisfactoriamente. 

La analogia con la POO es que la imagen es una clase y el contenedor una instancia de esa clase. 

    docker images

![](https://i.imgur.com/fOFBESM.png)

Aqui estan las imagenes que hemos venido usando en el curso. La columna TAG es algo asi como la version de la imagen. Docker por defecto descarga la que tenga el tag *latest* a menos que se lo especifiquemos, como fue el caso con *mongo*

### ¿De donde descarga DOcker las imagenes?

De *docker hub*: https://hub.docker.com/

Busquemos una imagen de Ubuntu

![](https://i.imgur.com/SZ5j5DS.png)

Observa que la version 22.04 tiene el tag de **latest** asi como otros tambien. 

### Pull

Vamos hacer el ejercicio de traer una version de ubuntu, pero que tenga otro tag.

    sudo docker pull ubuntu:bionic 

![](https://i.imgur.com/Lo4cyFq.png)

Y efectivamente, ya aparece:



## Construyendo una imagen propia

El proceso de creacion de imagenes esta basado en un archivo llamado *Dockerfile*, el cual describe lo que pasa cuando se crea la imagen,  y en conjunto con el comando *build* se contruira una imagen. 

Crearemos un nuevo directorio, llamado *imagenes*, e ingresaremos al mismo:

    mkdir imagenes
    cd imagenes

Crear un archivo *Dockerfile* dentro de imagenes:

    FROM ubuntu:latest

    RUN touch /usr/src/hola-platzi.txt

Todos los *Dockerfile* comienzan con FROM, porque partimos de algo, en este caso de la imagen *ubuntu:latest* que se encuentra en nuestro disco.

Y dado que tenemos un punto de partida, hagamos algo mas. Ese comando creara un nuevo archivo llamado *hola-platzi.txt*

Este comando RUN se ejecutara en tiempo de *build*, es decir al momento de construir la imagen. El comando *build* con la opcion -t indica el tag, y para seguir la nomenclaura de docker, es decir una imagen de ubuntu, y su version. Y por ultimo, el contexto de *build*, es decir, a que parte del disco tiene acceso el proceso mientras se ejecuta y crea la imagen. De ahi el punto. de directorio actual. 

    docker build -t ubuntu:platzi .

![](https://i.imgur.com/g90QTc4.png)




Cada instrucion que se coloque en el *Dockerfile* crear una nueva capa o layer. Verificamos que este:

    docker images

### Crear un nuevo contenedor con esa imagen y verificar que este el archivo hola-platzi.txt

    sudo docker run --name osos -d ubuntu:platzi tail -f /dev/null

    sudo docker exec -it osos bash

Y efectivamente el archivo se encuentra en esta nuevo contenedor basado en la imagen:

![](https://i.imgur.com/7FJ2eGB.png)


### Publicar imagen con push

Lo primero es login desde la consola. Yo configure un token, y lo guarde, igualmente pude acceder.

    docker login

Entonces, lo primero es usar el comando *tag*:

*carlcri/ubuntu:platzi* es la nomenclatura estandar de docker para mostrar quien es el dueño de esa imagen, que software contiene, y que version. 

    docker tag ubuntu:platzi carlcri/ubuntu:platzi

![](https://i.imgur.com/wrqUi77.png)



Y comprobamos hay un nuevo tag, pero curiosamente no se esta usando mas espacio en disco, para poderlo publicar

![](https://i.imgur.com/QgdTQKL.png)

Al agregar un nuevo tag a una imagen ya existente, no se esta creando una nueva imagen, sin ocupar mas espacio en disco.

Y ahora el push:

    sudo docker push carlcri/ubuntu:platzi

Observo hay un nuevo repositorio con el nombre de la imagen, y tambien un tag

![](https://i.imgur.com/Kv3OgJo.png)

La imagen que acabamos de publicar son publicas, asi que hay que tener cuidado no publicar nada sensible

## El sistema de capas

Iremos a *docker hub* y buscaremos el repositorio publico de ubuntu. Buscar la imagen de Ubuntu en Docker HUb, y examinemos el *DockerFile* de la version *latest*. Bueno por mas de que lo busque, no encontre el famoso docker file para ubuntu. 😞

Igualmente, muetro la explicacion:

Siempre se empieza de un *FROM*, y a medida que van corriendo otros comandos, como los que se muestran en rojo, se van creando capas

![](https://i.imgur.com/nDSPJiC.png)


### Uso del comando history.

Si estamos usando una imagen cuyo *docker file* no conocemos, se puede de esta manera:

    docker history ubuntu:platzi 

![](https://i.imgur.com/2BiM32s.png)

Se observa tenemos varias capas, e indicandonos cuanto pesa cada capa. Incluso, esta la instruccion que le colocamos anteriormente para crear un archivo. 

Sin embargo, es un poco dificil interpretar la salida del comando anterior, por lo que hay otra herramienta.

### dive

https://github.com/wagoodman/dive

Illendo al repositorio, la instalaremos:

    wget https://github.com/wagoodman/dive/releases/download/v0.9.2/dive_0.9.2_linux_amd64.deb
    sudo apt install ./dive_0.9.2_linux_amd64.deb

Y una vez instalada, analizamos una imagen en particular

    sudo dive ubuntu:platzi

![](https://i.imgur.com/zEa5nUr.png)

Para movernos a la derecha con TAB, y para regresar con TAB, 

### ¿Sera posible modificar una capa creada anteriormente?

Crearemos una nueva capa, modificando el *docker file*, borrando el archivo creamos anteriormente :

    RUN rm /usr/src/hola-platzi.txt

Ahora ejecutemos *docker build*, con el, mismo tag. Recuerda ejecutarlo estando en el folder *imagenes*

    sudo docker build -t ubuntu:platzi .

Y analizemos, nuevamente con *dive*:

    dive ubuntu:platzi

Mira que la capa anteriormente creada, no la modifico, creo una nueva. 

#### Conclusion

![](https://i.imgur.com/KxAsGI2.png)

Las capas son el corazon de las imagenes, una imagen simplemente es un nombre que apunta a una capa, y asi sucesivamente.

Las capas son inmutables, y cuando borramos un archivo que creamos en una capa anterior, en realidad esas dos capas existen, y desperdiciamos espacio. 

Cuando creamos un contenedo a partir de una imagen, tenemos acceso a la ultima capa de dicha imagen. Todos los cambios que hagamos sobre el contenedor, se haran en una capa, pero mutable, que se puede cambiar. 

Importante, las capas de la imagen no pueden ser cambiadas ni por el contenedor, no por nosotros mismos una vez las hallamos creado. 

Hay una forma de persistir esta capa mutable, que no se usa mucho, y es con comando *commit*